In [1]:
from uncertainties import unumpy, ufloat
import numpy as np
import matplotlib.pyplot as plt
import pywapor.et_look_v2_v3 as ETLook
import xarray as xr
import os
import datetime as dt
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
import tqdm
%run ../Functions.ipynb

/home/user/miniforge3/envs/phd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<>:31: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_96550/2858539060.py:31: SyntaxWarning: invalid escape sequence '\c'
  text = f'$y = {m:.2f} \cdot x + {b:.2f}$'


In [2]:
# Create a list of datestrings

# Start and end dates
# start_date = datetime(2018, 1, 1)
start_date = dt.datetime(2018, 1, 1)
end_date = dt.datetime(2022, 12, 31)

# List to store the formatted dates
date_strings = []

# Generate dates and format them as strings
current_date = start_date
while current_date <= end_date:
    date_strings.append(current_date.strftime('%Y-%m-%d'))
    current_date += dt.timedelta(days=1)

In [3]:
def sampling_LHS(es_mean_dts,es_std_dts,num_simulations=100,seed=42):
    # GET MIN MAX 
    # get daily mean and spread
    es_mean=es_mean_dts.resample(time='1D').mean().isel(time=0)
    es_std=es_std_dts.resample(time='1D').mean().isel(time=0)
    
    # calc 3-hourly RH
    t2m_k=da_to_uarray(es_mean_dts.t2m,es_std_dts.t2m)
    d2m_k=da_to_uarray(es_mean_dts.d2m,es_std_dts.d2m)
    rh=calc_RH(d2m_k,t2m_k,unit='K',source=1)
    
    ## get daily mean and spread of RHmin, RHmax
    tmin=t2m_k.min(axis=0)
    tmax=t2m_k.max(axis=0)
    rhmin=rh[2:7].min(axis=0)
    rhmax=rh[2:7].max(axis=0)
    # INPUT DATA    
    ds={}
    ## solar radiation
    ds["ra_24_J"] = get_LHS_simulations(es_mean.ssrd,es_std.ssrd,num_simulations)
    ds["ra_24"] = ds["ra_24_J"]/(3*3600) #convert daily average 3hourly J/m2 to total daily W/m2
    
    ## air temperature [K]
    ds["t_air_k_24"] = get_LHS_simulations(es_mean.t2m,es_std.t2m,num_simulations)
    ds["t_air_min_k_24"] = get_LHS_simulations(unumpy.nominal_values(tmin),
                                               unumpy.std_devs(tmin),num_simulations)
    ds["t_air_max_k_24"] = get_LHS_simulations(unumpy.nominal_values(tmax),
                                               unumpy.std_devs(tmax),num_simulations)
    ## air pressure [Pa]
    ds["p_air_24_pa"] = get_LHS_simulations(es_mean.sp,es_std.sp,num_simulations)
    
    ## windspeed at 10m [m/s]
    ds["u10m_24"] = get_LHS_simulations(es_mean.u10,es_std.u10,num_simulations)
    ds["v10m_24"] = get_LHS_simulations(es_mean.v10,es_std.v10,num_simulations)
    
    ## relative humidity or specific humidity or dew-point temperature
    # ds["qv_24"] = unumpy.uarray([[1, 2],[5, 3]], [[0.05, 0.004],[0.01, 0.002]])
    ds["rh_min_24"]=get_LHS_simulations(unumpy.nominal_values(rhmin),
                                               unumpy.std_devs(rhmin),num_simulations)
    ds["rh_max_24"]=get_LHS_simulations(unumpy.nominal_values(rhmax),
                                               unumpy.std_devs(rhmax),num_simulations)
    ds["t_dew_k_24"] = get_LHS_simulations(es_mean.d2m,es_std.d2m,num_simulations)
    
    # STATIC INPUT
    da_temp = es_mean.t2m
    ## day of the year
    date_obj = datetime.datetime.strptime(date, '%Y-%m-%d')
    day_of_year = date_obj.timetuple().tm_yday
    ds["doy"] = get_deterministic_arrays(day_of_year*np.ones_like(da_temp.values),num_simulations)
    
    ## latitude
    latitudes = da_temp['latitude']
    latitude_array = xr.DataArray(latitudes, dims=['latitude'])
    latitude_grid = latitude_array.broadcast_like(da_temp) # Use xarray broadcasting to create an array where each element of the array is the latitude value
    ds["y"] = get_deterministic_arrays(latitude_grid.values,num_simulations)
    
    # PARAMETERS
    ds["vp_slope"]= 0.14
    ds["vp_offset"]= 0.34
    ds["lw_slope"]= 1.35
    ds["lw_offset"]= -0.35
    ds["r0_grass"] = 0.23
    # ds["k_RS"] = 0.16 # correction for air mass FAO-56 Equ50
    return ds

In [ ]:
folder = r'./data/ERA5_Ensemble'
save_folder = r'./data/ERA5-RET_MCM500'
shapefile_path = r"./data/WaPOR_L1_v2_mask.shp"
seed = 42
np.random.seed(seed)
num_simulations = 500
mask=shapefile_path
for date in tqdm.tqdm(date_strings):
    es_mean_fhs = glob.glob(os.path.join(folder, f'ensemble_mean_{date}*.nc'))
    es_mean_dts = xr.open_mfdataset(es_mean_fhs)    
    es_std_fhs = glob.glob(os.path.join(folder, f'ensemble_spread_{date}*.nc'))
    es_std_dts = xr.open_mfdataset(es_std_fhs)
    da_temp=es_mean_dts.t2m[0]
    ds = sampling_LHS(es_mean_dts,es_std_dts,num_simulations=num_simulations,seed=seed)
    ds_out=calc_RET_FAO(ds,ret_only=True) 
    attrs={'variable':'RET (mm/day)',
                          'method':'Monte Carlo',
                          'sampling':'LHS',
                          'n_samples':num_simulations,
                          'function': 'calc_RET_FAO',
                          'input':'ERA5 ensemble mean and spread',
                          'seed':seed
                         }
    
    # save uarray(mean,std) of RET
    save_ds = xr.Dataset({
        'mean': xr.DataArray(np.nanmean(ds_out,axis=0),
                             dims=da_temp.dims,
                             coords=da_temp.coords,
                             attrs=attrs),
         'std': xr.DataArray(np.nanstd(ds_out,axis=0),
                             dims=da_temp.dims,
                             coords=da_temp.coords,
                             attrs=attrs),       
         'p5': xr.DataArray(np.nanpercentile(ds_out,5,axis=0),
                             dims=da_temp.dims,
                             coords=da_temp.coords,
                             attrs=attrs),
         'p95': xr.DataArray(np.nanpercentile(ds_out,95,axis=0),
                             dims=da_temp.dims,
                             coords=da_temp.coords,
                             attrs=attrs),        
    })   
    #mask
    amask = create_mask(save_ds, mask)
    save_ds = save_ds.where(amask, other=np.nan) 
    # 
    date_array = [np.datetime64(date)]
    new_dim_coord = xr.DataArray(date_array, dims='time')    
    save_ds = save_ds.assign_coords(time=new_dim_coord)
    results = save_ds.compute()
    filepath=os.path.join(save_folder,os.path.basename(date+'.nc'))
    results.to_netcdf(filepath)       

  0%|                                                                     | 0/1826 [00:00<?, ?it/s]/tmp/ipykernel_96550/2638014781.py:85: RuntimeWarning: invalid value encountered in sqrt
  (ds["vp_offset"]-ds["vp_slope"]*(ds['vp_24']**(1/2)))*\
/tmp/ipykernel_96550/1955040965.py:25: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  xarray_dataset.dims['longitude'], xarray_dataset.dims['latitude']
/tmp/ipykernel_96550/1955040965.py:28: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  width = int(xarray_dataset.dims['longitude'])
/tmp/ipykernel_96550/1955040965.py:29: FutureWarning: The return typ